In [1]:
!pip install -q "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.read_csv('/content/math_python_code.csv')
df = dataset_df.sample(frac=1).reset_index(drop=True)
df.head()

,Question,Python Code
0,Find the derivative of f(x) = x**3 + x,import sympy as sp\nx = sp.symbols('x')\nexpre...
1,Find the area of a square with side 6,s = 6\narea = s**2\nprint(area)
2,Solve the differential equation: Eq(Derivative...,"import sympy as sp\nx, f = sp.symbols('x f')\n..."
3,What is the sum of 1 and 4?,a = 1\nb = 4\nresult = a + b\nprint(result)
4,Find the derivative of f(x) = x**2,import sympy as sp\nx = sp.symbols('x')\nexpre...


In [10]:
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Math question:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"

PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
  intro=INTRO_BLURB,
  instruction_key=INSTRUCTION_KEY,
  instruction="{instruction}",
  input_key=INPUT_KEY,
  input="{input}",
  response_key=RESPONSE_KEY,
  response="{response}",
  end_key=END_KEY
)

In [11]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['Question', 'Python Code'],
    num_rows: 48
})

In [12]:
def apply_prompt_template(examples):
  instruction = "Write the python code for solve the given math question"
  context = examples.get("Question")
  response = examples["Python Code"]

  if context:
    full_prompt = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
  else:
    full_prompt = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
  return { "text": full_prompt }

dataset = dataset.map(apply_prompt_template)


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [13]:
print(dataset["text"][0])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write the python code for solve the given math question

Math question:
Find the derivative of f(x) = x**3 + x

### Response:
import sympy as sp
x = sp.symbols('x')
expression = x**3 + x
derivative = sp.diff(expression, x)
print(derivative)

### End


In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=bnb_config,
    device_map="cuda:0",
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj'] # Choose all linear layers from the model
)

In [16]:
from transformers import TrainingArguments

output_dir = "results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 5
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 30
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    ddp_find_unused_parameters=False,
)


In [17]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [18]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [19]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.997400
2,1.684600
3,1.457200
4,1.360000
5,1.062300
6,0.817800
7,0.690300
8,0.582400
9,0.486000
10,0.414600


TrainOutput(global_step=30, training_loss=0.45404042104880016, metrics={'train_runtime': 94.96, 'train_samples_per_second': 5.055, 'train_steps_per_second': 0.316, 'total_flos': 1414353502273536.0, 'train_loss': 0.45404042104880016, 'epoch': 10.0})

In [20]:
from transformers import pipeline

In [28]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write the python code for solve the given math question

Math question:
In a class of 30 pupils, 18 are boys. What is the percentage of boys in the class?

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write the python code for solve the given math question

Math question:
In a class of 30 pupils, 18 are boys. What is the percentage of boys in the class? 

### Response:
import math
boy_count = 18
class_size = 30
percentage = math.floor(boy_count / class_size * 100)
print(percentage)


None


In [25]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write the python code for solve the given math question

Math question:
What is the area and perimeter of a rectangle with a length of 8 cm and a width of 4 cm?

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write the python code for solve the given math question

Math question:
What is the area and perimeter of a rectangle with a length of 8 cm and a width of 4 cm? 

### Response:
s = 8
w = 4
area = s * w
perimeter = 2 * (s + w)
print(area, perimeter)


None


In [29]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)

prompt = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write the python code for solve the given math question

Math question:
2x+4y=20 and 5x+8y=30 what is the x,y values

### Response:
"""


result = pipe(prompt)
generated_text = result[0]['generated_text']
print(print(generated_text.split("### End")[0]))


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write the python code for solve the given math question

Math question:
2x+4y=20 and 5x+8y=30 what is the x,y values

### Response:
import sympy as sp
x, y = sp.symbols('x y')
equation1 = 2*x + 4*y == 20
equation2 = 5*x + 8*y == 30
solution_set = sp.dsolve(equation1 + equation2)
print(solution_set)
x, y_value = solution_set
print(x, y_value)


None
